> ### EEE4423: Deep Learning Lab

# Final Project: Long-tail Visual Recognition for Image Classification

<h4><div style="text-align: right"> Due date: June 24, 2022.  </div> <br>
<div style="text-align: right"> Please upload your file @ LearnUs and submit via e-mail by 2 PM in the form of [ID_Name_project.ipynb]. </div></h4>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/딥러닝 실험/project

In [3]:
import os
import os.path as osp
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from misc.project.utils import resnet18, IMBALANCECIFAR10, IMBALANCECIFAR100, compute_accuracy
from torch.nn.parameter import Parameter

import warnings
warnings.filterwarnings("ignore", category=UserWarning) ## 경고 무시

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
print('STEP 6: INSTANTIATE OPTIMIZER CLASS')

DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step'] # 질문
IMB_FACTOR = 0.1 #[0.1, 0.01]
SAVE_DIR = 'logs/LWS/CIFAR10/exp-0.1' 

LR = 0.1
BATCH_SIZE = 128 
MOMENTUM = 0.9
WEIGHT_DECAY = 2e-4
EPOCHS = 100

STEP 6: INSTANTIATE OPTIMIZER CLASS


In [6]:
os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [7]:
print('STEP 1: LOADING DATASET')

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

if DATASET == 'CIFAR10':
    train_dataset_0 = IMBALANCECIFAR10(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='../dataset/project', train=False, download=True, transform=transform_test)
elif DATASET == 'CIFAR100':
    train_dataset_0 = IMBALANCECIFAR100(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, train=True, download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR100(root='../dataset/project', train=False, download=True, transform=transform_test)

cls_num_list = train_dataset_0.get_cls_num_list() # 각 class에 몇개가 들어있는지 나타냄 
print('cls num list:')
print(cls_num_list)
num_classes = len(cls_num_list)
print(num_classes)

STEP 1: LOADING DATASET
Files already downloaded and verified
Files already downloaded and verified
cls num list:
[500, 488, 477, 466, 455, 445, 434, 424, 415, 405, 396, 387, 378, 369, 361, 352, 344, 336, 328, 321, 314, 306, 299, 292, 286, 279, 273, 266, 260, 254, 248, 243, 237, 232, 226, 221, 216, 211, 206, 201, 197, 192, 188, 183, 179, 175, 171, 167, 163, 159, 156, 152, 149, 145, 142, 139, 135, 132, 129, 126, 123, 121, 118, 115, 112, 110, 107, 105, 102, 100, 98, 95, 93, 91, 89, 87, 85, 83, 81, 79, 77, 75, 74, 72, 70, 69, 67, 66, 64, 63, 61, 60, 58, 57, 56, 54, 53, 52, 51, 50]
100


In [8]:
from torch.utils.data import Dataset, DataLoader, random_split
torch.manual_seed(torch.initial_seed())
generator = torch.Generator()
generator.manual_seed(0)
dataset_size = len(train_dataset_0)
validation_size = int(dataset_size * 0.1) if not int(dataset_size * 0.1)==0 else 1 
train_size = dataset_size - validation_size
test_size = 0

train_dataset, validation_dataset, test_dataset_0 = random_split(train_dataset_0, [train_size, validation_size, test_size],generator=generator)

print(len(train_dataset_0))
print(len(train_dataset))
print(len(validation_dataset))
print(len(test_dataset_0))

19573
17616
1957
0


In [9]:
print('STEP 2: MAKING DATASET ITERABLE')
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, drop_last=False)

val_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, drop_last=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=100, shuffle=False,
    num_workers=4, )

STEP 2: MAKING DATASET ITERABLE


In [10]:
print('STEP 3: CREATE MODEL CLASS (VGG16)')

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)
        self.scales = Parameter(torch.ones(num_classes))
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
        # LWS 구현
        pred *= self.scales
#         print(pred.size()) # torch.Size([128, 10])
        return pred

STEP 3: CREATE MODEL CLASS (VGG16)


In [11]:
print('STEP 4: INSTANTIATE MODEL CLASS')

model = ResNet18(num_classes)
model = model.cuda()

print('STEP 5: INSTANTIATE LOSS CLASS')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

STEP 4: INSTANTIATE MODEL CLASS
STEP 5: INSTANTIATE LOSS CLASS


In [12]:
torch.cuda.empty_cache()

In [36]:
for epoch in range(EPOCHS):
    
    loss_history = []
    model.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()

        pred = model(image)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    topk_acc, head_acc, tail_acc = compute_accuracy(val_loader, model)
    loss_mean = np.mean(loss_history)
    scheduler.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))


Epoch: [001] 	 Loss 3.6755 	 Acc 47.02 	 AccHead 50.56 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5171 	 Acc 44.52 	 AccHead 47.88 	 AccTail 0.00
Epoch: [003] 	 Loss 1.3501 	 Acc 58.31 	 AccHead 62.71 	 AccTail 0.00
Epoch: [004] 	 Loss 1.2167 	 Acc 57.26 	 AccHead 61.58 	 AccTail 0.00
Epoch: [005] 	 Loss 1.1790 	 Acc 59.84 	 AccHead 64.35 	 AccTail 0.00
Epoch: [006] 	 Loss 1.1295 	 Acc 57.02 	 AccHead 61.32 	 AccTail 0.00
Epoch: [007] 	 Loss 1.0945 	 Acc 62.18 	 AccHead 66.87 	 AccTail 0.00
Epoch: [008] 	 Loss 1.0665 	 Acc 61.85 	 AccHead 66.52 	 AccTail 0.00
Epoch: [009] 	 Loss 1.0522 	 Acc 64.76 	 AccHead 69.64 	 AccTail 0.00
Epoch: [010] 	 Loss 1.0113 	 Acc 64.92 	 AccHead 69.82 	 AccTail 0.00
Epoch: [011] 	 Loss 0.9789 	 Acc 65.08 	 AccHead 69.99 	 AccTail 0.00
Epoch: [012] 	 Loss 0.9511 	 Acc 64.92 	 AccHead 69.82 	 AccTail 0.00
Epoch: [013] 	 Loss 0.9481 	 Acc 67.74 	 AccHead 72.85 	 AccTail 0.00
Epoch: [014] 	 Loss 0.9192 	 Acc 67.02 	 AccHead 72.07 	 AccTail 0.00
Epoch: [015] 	 Loss 

In [37]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': epoch},
    osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(epoch+1))
)

In [13]:
## 
SAVE_DIR = 'logs/LWS/CIFAR10/exp-0.1' 
num_classes = 10
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 71.20 	 AccHead 77.56 	 AccTail 64.84


In [14]:
## 
SAVE_DIR = 'logs/LWS/CIFAR10/exp-0.01' 
num_classes = 10
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 51.28 	 AccHead 74.60 	 AccTail 27.96


In [15]:
## 
SAVE_DIR = 'logs/LWS/CIFAR10/step-0.1' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 64.35 	 AccHead 82.28 	 AccTail 46.42


In [16]:
## 
SAVE_DIR = 'logs/LWS/CIFAR10/step-0.01' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 47.57 	 AccHead 88.52 	 AccTail 6.62


여기서 부터는 CIFAR100 이다.

In [25]:
## 
SAVE_DIR = 'logs/LWS/CIFAR100/exp-0.1' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 35.45 	 AccHead 44.46 	 AccTail 26.44


In [27]:
## 
SAVE_DIR = 'logs/LWS/CIFAR100/exp-0.01' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 23.52 	 AccHead 37.24 	 AccTail 9.80


In [28]:
## 
SAVE_DIR = 'logs/LWS/CIFAR100/step-0.1' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 33.40 	 AccHead 50.70 	 AccTail 16.10


In [29]:
## 
SAVE_DIR = 'logs/LWS/CIFAR100/step-0.01' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep100.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 26.72 	 AccHead 52.08 	 AccTail 1.36


이건 CIFAR100에서 경향성을 알기 위해 epoch200까지 돌린 결과이다. 

In [30]:
## 
SAVE_DIR = 'logs/LWS/CIFAR100/exp-0.1' 
model = ResNet18(num_classes)
model.load_state_dict(torch.load(
    osp.join(SAVE_DIR, 'ep200.pth')
)['model'])
model = model.cuda()
topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

Acc 40.31 	 AccHead 50.72 	 AccTail 29.90


실행 시간을 위한 코드

In [13]:
import time 

train_start_time = time.time()

for epoch in range(EPOCHS):
    loss_history = []
    model.train()
    for batch_index, data in enumerate(train_loader):
        image, target = data
        image, target = image.cuda(), target.cuda()

        pred = model(image)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

    topk_acc, head_acc, tail_acc = compute_accuracy(val_loader, model)
    loss_mean = np.mean(loss_history)
    scheduler.step()

    print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))

train_end_time = time.time()

print(train_end_time-train_start_time)

Epoch: [001] 	 Loss 4.2546 	 Acc 10.83 	 AccHead 13.18 	 AccTail 3.25
Epoch: [002] 	 Loss 3.7929 	 Acc 13.75 	 AccHead 16.25 	 AccTail 5.63
Epoch: [003] 	 Loss 3.5781 	 Acc 14.10 	 AccHead 17.06 	 AccTail 4.55
Epoch: [004] 	 Loss 3.4303 	 Acc 17.73 	 AccHead 20.13 	 AccTail 9.96
Epoch: [005] 	 Loss 3.2726 	 Acc 20.34 	 AccHead 22.47 	 AccTail 13.42
Epoch: [006] 	 Loss 3.1339 	 Acc 23.45 	 AccHead 26.96 	 AccTail 12.12
Epoch: [007] 	 Loss 3.0044 	 Acc 24.17 	 AccHead 28.09 	 AccTail 11.47
Epoch: [008] 	 Loss 2.8942 	 Acc 23.91 	 AccHead 27.96 	 AccTail 10.82
Epoch: [009] 	 Loss 2.7981 	 Acc 24.43 	 AccHead 28.36 	 AccTail 11.69
Epoch: [010] 	 Loss 2.6817 	 Acc 28.26 	 AccHead 31.77 	 AccTail 16.88
Epoch: [011] 	 Loss 2.5773 	 Acc 27.85 	 AccHead 31.44 	 AccTail 16.23
Epoch: [012] 	 Loss 2.4892 	 Acc 29.18 	 AccHead 32.37 	 AccTail 18.83
Epoch: [013] 	 Loss 2.3992 	 Acc 31.89 	 AccHead 35.12 	 AccTail 21.43
Epoch: [014] 	 Loss 2.2948 	 Acc 34.34 	 AccHead 38.93 	 AccTail 19.48
Epoch: [01

In [14]:
train_start_time = time.time()

topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)

print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

train_end_time = time.time()

print(train_end_time-train_start_time)

Acc 34.27 	 AccHead 43.88 	 AccTail 24.66
4.967448949813843
